In [1]:
import os
import random
import tensorflow as tf
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, AdamWeightDecay,DataCollatorForSeq2Seq

2024-04-25 16:15:37.222651: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 16:15:37.222803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 16:15:37.383121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tf.__version__

'2.15.0'

In [3]:
import numpy as np
import pandas as pd
data = np.load('/kaggle/input/sp-train/SP-train.npy', allow_pickle=True)

In [4]:
# from sklearn.model_selection import train_test_split

# train_data, test_val_data = train_test_split(data, test_size=102, random_state=42)
# test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

In [5]:
# train_l = []
# test_l = []
# val_l = []
# for i in train_data:
#     for j in range(4):
#         d = {}
#         d['sentence1'] = i['question']
#         d['sentence2'] = i['choice_list'][j]
#         if(i['label']==j):
#             d['label1'] = 'entailment'
#         else:
#             d['label1'] = 'other'
#         train_l.append(d)
# for i in test_data:
#     for j in range(4):
#         d = {}
#         d['sentence1'] = i['question']
#         d['sentence2'] = i['choice_list'][j]
#         if(i['label']==j):
#             d['label1'] = 'entailment'
#         else:
#             d['label1'] = 'other'
#         test_l.append(d)
# for i in val_data:
#     for j in range(4):
#         d = {}
#         d['sentence1'] = i['question']
#         d['sentence2'] = i['choice_list'][j]
#         if(i['label']==j):
#             d['label1'] = 'entailment'
#         else:
#             d['label1'] = 'other'
#         val_l.append(d)

In [6]:
# import pandas as pd
# train_df = pd.DataFrame(train_l)
# test_df = pd.DataFrame(test_l)
# valid_df = pd.DataFrame(val_l)

In [7]:
l = []
for i in data:
    for j in range(4):
        d = {}
        d['sentence1'] = i['question']
        d['sentence2'] = i['choice_list'][j]
        if(i['label']==j):
            d['label1'] = 'entailment'
        else:
            d['label1'] = 'other'
        l.append(d)
df = pd.DataFrame(l)

In [8]:
tf.test.is_gpu_available()

True

In [9]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [11]:
model = TFAutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [12]:
def preprocess_function(dataframe):
    inputs = []
    targets = []
    for i in range(dataframe.shape[0]):
        sentence1 = dataframe['sentence1'][i]
        sentence2 = dataframe['sentence2'][i]
        input = "rte sentence1: %s  sentence2: %s" % (sentence1, sentence2)
        target = "%s" % dataframe['label1'][i]
        inputs.append(input)
        targets.append(target)
    tokenized_data = tokenizer(inputs, text_target = targets)
    return tokenized_data

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='google-t5/t5-small', return_tensors="tf")

In [14]:
 df.shape

(2028, 3)

In [15]:
train_df = df.iloc[:1620,:]
valid_df = df.iloc[1620:1824,:]
test_df = df.iloc[1824:2028,:]

In [16]:
valid_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df.reset_index(drop=True, inplace=True)

In [17]:
test_df.head(12)

,sentence1,sentence2,label1
0,"If a monkey, a squirrel, and a bird are racing...",The squirrel.,other
1,"If a monkey, a squirrel, and a bird are racing...",The monkey.,other
2,"If a monkey, a squirrel, and a bird are racing...",The bird.,other
3,"If a monkey, a squirrel, and a bird are racing...",None of above.,entailment
4,"Who will grab the banana first if a monkey, a ...",The bird.,other
5,"Who will grab the banana first if a monkey, a ...",The squirrel.,other
6,"Who will grab the banana first if a monkey, a ...",The monkey.,other
7,"Who will grab the banana first if a monkey, a ...",None of above.,entailment
8,A rooster and a bull were expecting their firs...,The rooster.,other
9,A rooster and a bull were expecting their firs...,The bull.,other


In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)
train_set = model.prepare_tf_dataset(
    Dataset.from_dict(preprocess_function(train_df)),
    shuffle=True,
    batch_size= 64,
    collate_fn=data_collator,
)

In [19]:
validation_set = model.prepare_tf_dataset(
    Dataset.from_dict(preprocess_function(valid_df)),
    shuffle=False,
    batch_size= 64,
    collate_fn=data_collator,
)

In [20]:
test_set = model.prepare_tf_dataset(
    Dataset.from_dict(preprocess_function(test_df)),
    shuffle=False,
    batch_size= 64,
    collate_fn=data_collator,
)

In [21]:
optimizer = AdamWeightDecay(learning_rate=3e-4, weight_decay_rate=0.01)
model.compile(optimizer = optimizer)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [22]:
!rm -rf /kaggle/working/*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
model.fit(x=train_set, validation_data=validation_set, epochs=10)
#callbacks=[early_stopping_callback]

Epoch 1/10
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
25/25 [==============================] - 122s 1s/step - loss: 0.9910 - val_loss: 0.2100
Epoch 2/10
25/25 [==============================] - 10s 383ms/step - loss: 0.2193 - val_loss: 0.2026
Epoch 3/10
25/25 [==============================] - 10s 384ms/step - loss: 0.2027 - val_loss: 0.2037
Epoch 4/10
25/25 [==============================] - 10s 381ms/step - loss: 0.1915 - val_loss: 0.2091
Epoch 5/10
25/25 [==============================] - 10s 387ms/step - loss: 0.1615 - val_loss: 0.1721
Epoch 6/10
25/25 [==============================] - 10s 386ms/step - loss: 0.1266 - val_loss: 0.1655
Epoch 7/10
25/25 [==============================] - 10s 390ms/step - loss: 0.0916 - val_loss: 0.1890
Epoch 8/10
25/25 [==============================] - 10s 386ms/step - loss: 0.0663 - val_loss: 0.2029
Epoch 9/10
25/25 [==============================] - 10s 

In [24]:


# loaded_model = tf.keras.models.load_model(keras_model_path)
# loaded_model.compile(optimizer = optimizer)

In [25]:
def generate_prediction(sentence1, sentence2):
    input = "rte sentence1: %s  sentence2: %s" % (sentence1, sentence2)
    tokenized_input = tokenizer(input, return_tensors="tf")
    tokenized_output = model.generate(tokenized_input['input_ids'])
    output = tokenizer.decode(tokenized_output[0], skip_special_tokens=True)
    return output

In [26]:
sentence1 = 'This church choir sings to the masses as they sing joyous songs from the book at a church.'
sentence2 = 'The church has cracks in the ceiling.'
generate_prediction(sentence1,sentence2)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
I0000 00:00:1714061972.690690     103 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


'other'

In [27]:
sentence2 = 'The church is filled with song.'
generate_prediction(sentence1,sentence2)

'other'

In [28]:
def generate_predictions_from_dataframe(dataframe):
    probs = []  # Array to store probabilities
    preds = []
    for i in range(dataframe.shape[0]):
        input_text = "rte sentence1: %s  sentence2: %s" % (dataframe['sentence1'][i], dataframe['sentence2'][i])
        tokenized_input = tokenizer(input_text, return_tensors="tf")
        outputs = model.generate(tokenized_input['input_ids'], return_dict_in_generate=True, output_scores=True)
        token_ids = outputs.sequences[0]
        scores = outputs.scores[0]

        # Decode the predicted token IDs into text
        output_text = tokenizer.decode(token_ids, skip_special_tokens=True)

        # Get the probabilities for each token
        token_probs = tf.nn.softmax(scores, axis=-1).numpy()

        # Calculate the probability for 'entailment' tag
        entailment_prob = token_probs[0][tokenizer.encode('entailment')[0]]
        
        probs.append(entailment_prob)  # Store the probability in the array
        preds.append((output_text, entailment_prob))
    return preds, probs


In [29]:
predictions, probabilities = generate_predictions_from_dataframe(test_df)
# for pred_text, entailment_prob in predictions:
#     print("Prediction Text:", pred_text)
#     print("Entailment Probability:", entailment_prob)

# Access the probabilities array
print("All Probabilities:", probabilities)


All Probabilities: [0.71580803, 0.9840553, 0.8620366, 0.5608817, 0.986026, 0.9479194, 0.9965467, 0.53929025, 0.9997485, 0.9877437, 0.0037504856, 0.08925641, 0.0021842371, 0.002245957, 0.011202734, 0.64818275, 0.001583594, 0.0013633955, 0.0020236543, 0.07248079, 0.0010069254, 0.0007010405, 0.0015185039, 0.060389068, 0.003396471, 0.07945524, 0.00016952792, 0.00809956, 0.16753806, 0.08587064, 0.0004745628, 0.018533802, 0.23169161, 0.96684587, 3.641081e-05, 0.003324434, 0.9419584, 0.00049727084, 2.6522948e-05, 0.002725263, 2.9968794e-05, 0.59081995, 0.00020656818, 0.0032025236, 0.00010726589, 9.6779906e-05, 7.0111375e-05, 0.0005906553, 0.0052687195, 0.29616192, 0.0031236927, 0.5175945, 0.29616192, 0.0052687195, 0.0031236927, 0.5175945, 0.00023723027, 0.3155882, 0.005674223, 0.030768637, 0.0020538531, 0.002012818, 0.0023436027, 0.17210011, 0.0019255115, 0.0017848157, 0.0015726177, 0.30999032, 0.0015806203, 0.0007564798, 0.00086247176, 0.14272062, 0.9999448, 0.09474164, 3.5337915e-05, 0.0002

In [30]:
import numpy as np

# Assume 'probabilities' is the array containing the probabilities
# Replace this line with your actual probabilities array

# Create an empty array to store the final predictions
final_predictions = []
count=0
count2=0

for i in range(0, len(probabilities), 4):
    current_probs = probabilities[i:i + 4]
    
    max_index = np.argmax(current_probs)
    
    for j in range(i, i + 4):
        if j == i + max_index:
            final_predictions.append('1')
            count+=1
#             print("e")
        else:
            final_predictions.append('0')
            count2+=1
#             print("other")
#     print()

# print(count)
# print(count2)
# print("Final Predictions:", final_predictions)


In [31]:
test_df['final_predictions'] = final_predictions
test_df.head()

/tmp/ipykernel_34/2679610233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['final_predictions'] = final_predictions


,sentence1,sentence2,label1,final_predictions
0,"If a monkey, a squirrel, and a bird are racing...",The squirrel.,other,0
1,"If a monkey, a squirrel, and a bird are racing...",The monkey.,other,1
2,"If a monkey, a squirrel, and a bird are racing...",The bird.,other,0
3,"If a monkey, a squirrel, and a bird are racing...",None of above.,entailment,0
4,"Who will grab the banana first if a monkey, a ...",The bird.,other,0


In [32]:
true_label = []
pred_label = []
for index, row in test_df.iterrows():
    if row['label1'] == 'entailment':
        true_label.append('1')
        pred_label.append(row['final_predictions'])

In [33]:
len(true_label),len(pred_label)

(51, 51)

In [34]:
correct_labels=0

for i in pred_label:
    if(i=='1'):
        correct_labels+=1

In [35]:
print(correct_labels/len(pred_label))

0.6862745098039216


In [36]:
# true_label,pred_label

In [37]:
from sklearn import metrics
metrics.accuracy_score(true_label,pred_label)

0.6862745098039216

In [38]:
keras_model_path = '/kaggle/working/fine_tuned.keras'
model.save(keras_model_path)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
